In [1]:
# comments and details to come !
import time
import numpy as np
import scipy as sp
import imageio
import os
from IPython.display import display, clear_output, HTML
import sys
os.environ['PYOPENCL_COMPILER_OUTPUT'] = '0'

from mcClass import *
import matplotlib.pyplot as plt
from matplotlib import animation, rc
animation.rcParams['animation.writer']='ffmpeg'

%matplotlib inline

In [3]:
fM = 2.0
fS = 1.5
octa = 1
th = 20
thS = 20
fT = 1.0/0.1
v = 2.0*np.array([np.cos(th),np.sin(th)])

MC = motionCloud(overSamp=4, N=128, chooseDev=0, show=1)
MC.mcKernel(fM, fS, th, thS, fT, v, octa)
MC.initGPU()
            

Correct parameters LifeTime = 0.100000 > 0.095113 



In [4]:
# First set up the figure, the axis, and the plot element we want to animate
fig01, axes01 = plt.subplots(nrows=1, ncols=1,figsize=(5, 5))
plt.close()

def init():
    return ()


def animate(i): 
    
    im0 = axes01.imshow(MC.getFrame(), cmap='gray')
    axes01.xaxis.set_visible(False)
    axes01.yaxis.set_visible(False)
    
    clear_output(wait=True)
    print 'It: %i'%i
    sys.stdout.flush()
    return (im0,)
            
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig01, animate, init_func=init, frames=100, interval=50, blit=True)
HTML(anim.to_html5_video()) 


It: 99


In [46]:
im = np.zeros((128,128,3,128), dtype=np.uint8)

for i in range(128):
    frame = MC.getFrame() + 127.5
    frame[frame>255]=255
    frame[frame<0]=0
    im[:,:,0,i] = np.uint8(frame)
    im[:,:,1,i] = np.uint8(frame)
    im[:,:,2,i] = np.uint8(frame)
    
wvid = imageio.get_writer('mc.avi', fps=25.0)

for i in range(128):
    wvid.append_data(im[:,:,:,i])
    
wvid.close()